# Compute Radiomics

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AustralianCancerDataNetwork/pydicer/blob/main/examples/Radiomics.ipynb)

In this example notebook we use [PyRadiomics](https://github.com/AIM-Harvard/pyradiomics) to
compute various type of radiomics features. We use some
[LCTSC](https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=24284539) data from the
Cancer Imaging Archive which has already been converted using PyDicer for demonstration purposes.

In [ ]:
try:
    from pydicer import PyDicer
except ImportError:
    !pip install pydicer
    from pydicer import PyDicer

from pathlib import Path

from pydicer.utils import fetch_converted_test_data

from pydicer.utils import load_object_metadata

## Fetch data

LCTSC data prepared for this example are downloaded and stored into a `testdata_lctsc` directory.
We will use this for our PyDicer working directory.

In [ ]:
working_directory = fetch_converted_test_data("./testdata_lctsc", dataset="LCTSC")

## Initialise PyDicer object

Using the working directory containing the LCTSC test data.

In [ ]:
pydicer = PyDicer(working_directory)

## Compute Default Radiomics

By default, PyDicer will compute only first-order radiomics features. Radiomics are computed for
each structure available in the dataset using the image data of images linked to those structures.

In [ ]:
pydicer.analyse.compute_radiomics()

## Fetch computed Radiomics

Use the [get_all_computed_radiomics_for_dataset](https://australiancancerdatanetwork.github.io/pydicer/analyse.html#pydicer.analyse.data.AnalyseData.get_all_computed_radiomics_for_dataset) function to fetch all radiomics features computed
in the last step.

The `.head()` function on a Pandas DataFrame output the first 5 rows for inspection.

In [ ]:
# Display the DataFrame of radiomics computed
df_radiomics = pydicer.analyse.get_all_computed_radiomics_for_dataset()
df_radiomics.head()

## Specify Radiomics to Compute

PyDicer used the popular `pyradiomics` library to compute radiomics. So, you may specify any
radiomics features provided in that library to be computed. See the [pyradiomics documentation for
a list of radiomics features
available](https://pyradiomics.readthedocs.io/en/latest/features.html).

In this example, we specify all `shape` features as well as `first-order` features to be computed.

In [ ]:
# Import pyradiomics feature modules
from radiomics import (
    firstorder,
    shape,
)

# Prepare a dict of features to compute grouped by class
first_order_features = firstorder.RadiomicsFirstOrder.getFeatureNames()
shape_features = shape.RadiomicsShape.getFeatureNames()
compute_radiomics = {
    "firstorder": [f for f in first_order_features if not first_order_features[f]],
    "shape": [f for f in shape_features if not shape_features[f]],
}

# Pass the dict to the compute the radiomics
pydicer.analyse.compute_radiomics(radiomics=compute_radiomics)

# Fetch the computed radiomics and output the first few rows
df_radiomics = pydicer.analyse.get_all_computed_radiomics_for_dataset()
df_radiomics.head()

You can also set a specific subset of features like this:

In [ ]:
# Specify features to compute grouped by class
compute_radiomics = {
    "firstorder": ["Maximum", "Minimum", "Mean", "Median"],
    "shape": ["SurfaceArea", "VoxelVolume"],
}

# Pass the dict to the compute the radiomics
pydicer.analyse.compute_radiomics(radiomics=compute_radiomics)

df_radiomics = pydicer.analyse.get_all_computed_radiomics_for_dataset()
df_radiomics.head()

## Track Metadata

When analysing your radiomic features, it may be useful to have certain metadata available from
either the image or structure set. You can specify which DICOM header tags to extract metadata for
and these will be stored alongside the radiomic feature values.

In the cell below, we recompute our radiomics and store the `PatientSex` header value from the
image series and the `StudyDate` value from the structure set.

In [ ]:
# Compute the radiomics specifying the meta data to keep
pydicer.analyse.compute_radiomics(
    radiomics=compute_radiomics,
    image_meta_data=["PatientSex"],
    structure_meta_data=["StudyDate"]
)

# Fetch the results and display the first rows
df_radiomics = pydicer.analyse.get_all_computed_radiomics_for_dataset()
df_radiomics.head()